<a href="https://colab.research.google.com/github/trdunsworth/eda2/blob/main/VA_NENA_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pandas numpy scipy matplotlib seaborn scikit-learn plotly statsmodels

In [3]:
!pip install pretty_errors

In [4]:
%matplotlib inline

import os
import sys
import numbers
import math
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import datetime as dt
import pandas as pd
import numpy as np
from scipy import stats as ss
from scipy.stats import normaltest as norm
from matplotlib import pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.graphics.gofplots import qqplot
import pretty_errors

In [5]:
pretty_errors.configure(
    separator_character="*",
    line_number_first=1,
    display_link=1,
    line_color=pretty_errors.BRIGHT_RED
    + "> "
    + pretty_errors.default_config.line_color,
    exception_color=pretty_errors.BRIGHT_MAGENTA,
    exception_arg_color=pretty_errors.CYAN,
    exception_file_color=pretty_errors.RED_BACKGROUND + pretty_errors.BRIGHT_WHITE,
    code_color="  " + pretty_errors.default_config.line_color,
    display_timestamp=1,
    timestamp_function=lambda: datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    lines_before=5,
    lines_after=2,
    stack_depth=1,
    display_locals=1,  # Enable the display of local variables
    line_length=80
)

In [6]:
"""
This code sets display and formatting parameters for the notebook
For example: floats, numbers with a decimal point, will only show 2 values behind the decimal
There is a column wrap at 80 characters and this will only display 8 columns of multicolumn output
"""
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.width', 80)
pd.set_option('display.max_columns', 12)

In [7]:
df = pd.read_csv("/content/drive/MyDrive/synthetic_date/synth.csv")